In [1]:
import os 

In [4]:
%pwd

'e:\\Rag-gen-ai-project\\FinanceMentor-AI'

In [3]:
os.chdir("../")

In [5]:
from langchain.document_loaders import DirectoryLoader,PyPDFLoader 
from langchain.text_splitter import RecursiveCharacterTextSplitter 

In [6]:
def load_pdf_files(data): 
    loader=DirectoryLoader(
        data, 
        glob="*.pdf", 
        loader_cls=PyPDFLoader
    )
    documents=loader.load()
    return documents 

In [7]:
def text_splitter(data): 
    text_split=RecursiveCharacterTextSplitter(chunk_size=2000,chunk_overlap=500)
    text_chunk=text_split.split_documents(data)
    return text_chunk

In [ ]:
extracted_data=load_pdf_files("Data/")
text_chunk=text_splitter(extracted_data)